## Chapter 9: Custom Resources and Lifecycle Hooks

In the previous chapters, you’ve used Aspire’s built‑in resource types—projects, containers, databases, etc.—to model your application. But what if you need a resource that isn’t one of the standard ones? Perhaps you need to run a custom setup script, wait for a third‑party service to be ready, or coordinate the startup order in a more fine‑grained way. Aspire’s extensibility model allows you to create **custom resources** and hook into the application lifecycle with **lifecycle hooks**.

In this chapter, you’ll learn:

- How to create a custom resource type.
- How to attach metadata using annotations.
- How to implement lifecycle hooks (`BeforeStart`, `AfterResourcesCreated`) to execute custom logic.
- How to run database migrations automatically before your API starts.
- How to model complex dependencies.

By the end, you’ll be able to extend Aspire’s orchestrator to fit any bespoke requirement.

---

### 9.1 Beyond Built‑in Resources

The built‑in resources (`AddProject`, `AddContainer`, `AddPostgres`, etc.) cover many common scenarios. However, you might need:

- A resource representing a custom tool or script.
- A resource that waits for an external condition (e.g., a cloud service to be provisioned).
- A resource that groups other resources together.
- A resource that performs an action (like database migration) rather than running continuously.

Aspire allows you to define your own resource types by implementing the `IResource` interface or deriving from `Resource`. You then add them to the application model using `AddResource`.

---

### 9.2 Creating a Custom Resource

Let’s create a custom resource that represents a **database migration** task. This resource won’t run continuously; instead, it will execute a migration script (using EF Core) and then complete. We want the API project to depend on this migration resource, so that the API starts only after migrations have run.

#### Step 1: Define the Custom Resource Class

Create a new class in the AppHost project (or a shared project) called `MigrationResource.cs`:

```csharp
using Aspire.Hosting.ApplicationModel;

namespace MyAspireApp.AppHost;

public class MigrationResource : Resource
{
    public MigrationResource(string name) : base(name)
    {
    }
}
```

This simple class derives from `Resource`, which gives it a name and the ability to have annotations. We’ll add more later.

#### Step 2: Add an Extension Method for Easy Addition

Create an extension method so we can write `builder.AddMigration("migration")` in a fluent way:

```csharp
using Aspire.Hosting.ApplicationModel;

namespace Aspire.Hosting;

public static class MigrationResourceBuilderExtensions
{
    public static IResourceBuilder<MigrationResource> AddMigration(
        this IDistributedApplicationBuilder builder,
        string name)
    {
        var resource = new MigrationResource(name);
        return builder.AddResource(resource)
                      .WithInitialState(new CustomResourceSnapshot
                      {
                          ResourceType = "Migration",
                          State = "Not started",
                          Properties = []
                      });
    }
}
```

The `WithInitialState` call sets an initial state for the dashboard, so the resource appears with a custom type and status.

#### Step 3: Use the Custom Resource in AppHost

In `Program.cs`, add the migration resource and make the API depend on it:

```csharp
var migration = builder.AddMigration("migration");

var apiService = builder.AddProject<Projects.MyAspireApp_ApiService>("apiservice")
    .WithReference(migration)   // This creates a dependency
    .WithReference(productsDb)
    .WithReference(messaging)
    .WithReference(blobs);
```

Now the API will not start until the migration resource is considered “running”. But our migration resource currently does nothing. We need to add behavior.

---

### 9.3 Lifecycle Hooks

Lifecycle hooks allow you to execute code at specific points during the application’s startup and shutdown. The main hooks are:

- **BeforeStart**: Executed before any resources are started. Good for validating configuration or preparing the environment.
- **AfterResourcesCreated**: Executed after all resources have been created (i.e., their underlying processes/containers are running). This is perfect for running one‑time setup tasks that depend on other resources.

To use lifecycle hooks, you attach an **annotation** that implements `ILifecycleHook` to your resource.

#### 9.3.1 Implementing a Lifecycle Hook

Create a class that implements `IDistributedApplicationLifecycleHook`. You can override methods like `BeforeStartAsync` and `AfterResourcesCreatedAsync`.

```csharp
using Aspire.Hosting.Lifecycle;
using Aspire.Hosting.ApplicationModel;

namespace MyAspireApp.AppHost;

public class MigrationLifecycleHook : IDistributedApplicationLifecycleHook
{
    public Task BeforeStartAsync(DistributedApplicationModel appModel, CancellationToken cancellationToken = default)
    {
        // You could validate something here
        return Task.CompletedTask;
    }

    public Task AfterResourcesCreatedAsync(DistributedApplicationModel appModel, CancellationToken cancellationToken = default)
    {
        // Here we can run migrations
        return Task.CompletedTask;
    }
}
```

But we need this hook to be specific to our `MigrationResource`. The `appModel` contains all resources. We can find our migration resource and then execute the migration logic. However, we also need to wait for the database resource to be ready (since migrations need a database). This is where annotations help.

#### 9.3.2 Attaching the Lifecycle Hook to the Resource

We don’t want the hook to run for every resource; we only want it to run for our specific migration resource. So we’ll create a custom annotation that holds the logic, and then attach it to the resource. Then, in a global lifecycle hook, we can look for resources with that annotation and execute them.

First, define an annotation interface:

```csharp
public interface IMigrationExecutor
{
    Task ExecuteMigrationAsync(CancellationToken cancellationToken);
}
```

Then create a class that implements this interface and also holds a reference to the resource and its dependencies.

```csharp
using Aspire.Hosting.ApplicationModel;

namespace MyAspireApp.AppHost;

public class MigrationExecutorAnnotation : IMigrationExecutor
{
    private readonly MigrationResource _resource;
    private readonly DistributedApplicationModel _model;

    public MigrationExecutorAnnotation(MigrationResource resource, DistributedApplicationModel model)
    {
        _resource = resource;
        _model = model;
    }

    public async Task ExecuteMigrationAsync(CancellationToken cancellationToken)
    {
        // Find the database connection string from the API service's references
        // We need to locate the API service that references this migration.
        // But a more robust way: have the migration resource reference the database directly.
        // Let's assume the migration resource also references the database.

        // For simplicity, we'll just run a command in the API project.
        // In reality, you'd run `dotnet ef database update` against the API project.

        var apiService = _model.Resources.OfType<ProjectResource>()
            .FirstOrDefault(r => r.Name == "apiservice");

        if (apiService == null)
        {
            throw new InvalidOperationException("API service not found");
        }

        // Get the database connection string from the API's environment
        // This is tricky because environment variables are set after resources start.
        // Better: have the migration resource reference the database and get its connection string directly.

        // We'll retrieve the connection string from the database resource.
        var dbResource = _model.Resources.OfType<PostgresDatabaseResource>()
            .FirstOrDefault(r => r.Name == "productsdb");

        if (dbResource == null)
        {
            throw new InvalidOperationException("Database resource not found");
        }

        // Get the connection string
        var connectionString = await dbResource.GetConnectionStringAsync(cancellationToken);

        // Run EF migration command
        var projectPath = apiService.GetProjectMetadata().ProjectPath;
        var workingDir = Path.GetDirectoryName(projectPath);

        var process = Process.Start(new ProcessStartInfo
        {
            FileName = "dotnet",
            Arguments = "ef database update --connection \"" + connectionString + "\"",
            WorkingDirectory = workingDir,
            RedirectStandardOutput = true,
            RedirectStandardError = true
        });

        await process.WaitForExitAsync(cancellationToken);
        if (process.ExitCode != 0)
        {
            var error = await process.StandardError.ReadToEndAsync();
            throw new InvalidOperationException($"Migration failed: {error}");
        }
    }
}
```

Now we need to attach this annotation to the migration resource. Update the extension method:

```csharp
public static IResourceBuilder<MigrationResource> AddMigration(
    this IDistributedApplicationBuilder builder,
    string name)
{
    var resource = new MigrationResource(name);
    var resourceBuilder = builder.AddResource(resource)
        .WithInitialState(new CustomResourceSnapshot
        {
            ResourceType = "Migration",
            State = "Pending",
            Properties = []
        });

    // We'll attach the executor later, after the model is built.
    // But we can't create the executor yet because we need the model.
    // Instead, we'll use a global lifecycle hook that finds this resource and runs its executor.

    return resourceBuilder;
}
```

We’ll create a global lifecycle hook that looks for all `MigrationResource` instances and runs their migration logic. But we need to pass the connection string etc. So we’ll create a separate hook that runs after resources are created.

#### 9.3.3 Creating a Global Lifecycle Hook

Create a class that implements `IDistributedApplicationLifecycleHook` and scans for migration resources:

```csharp
using Aspire.Hosting.Lifecycle;
using Aspire.Hosting.ApplicationModel;

namespace MyAspireApp.AppHost;

public class MigrationLifecycleHook : IDistributedApplicationLifecycleHook
{
    public async Task AfterResourcesCreatedAsync(DistributedApplicationModel appModel, CancellationToken cancellationToken = default)
    {
        // Find all migration resources
        var migrationResources = appModel.Resources.OfType<MigrationResource>().ToList();
        foreach (var migrationResource in migrationResources)
        {
            // For each, we need to find the database it should migrate.
            // We'll assume the migration resource references the database via annotation.
            // Let's get the connection string from the database resource that is referenced.
            var dbReference = migrationResource.Annotations.OfType<ReferenceAnnotation>()
                .FirstOrDefault(r => r.Resource is PostgresDatabaseResource);
            if (dbReference == null)
            {
                throw new InvalidOperationException($"Migration resource {migrationResource.Name} does not reference a database.");
            }

            var dbResource = (PostgresDatabaseResource)dbReference.Resource;
            var connectionString = await dbResource.GetConnectionStringAsync(cancellationToken);

            // Now we need to know which project to run migrations against.
            // We could also store a reference to the project in an annotation.
            var projectReference = migrationResource.Annotations.OfType<ReferenceAnnotation>()
                .FirstOrDefault(r => r.Resource is ProjectResource);
            if (projectReference == null)
            {
                throw new InvalidOperationException($"Migration resource {migrationResource.Name} does not reference a project.");
            }

            var projectResource = (ProjectResource)projectReference.Resource;
            var projectPath = projectResource.GetProjectMetadata().ProjectPath;
            var workingDir = Path.GetDirectoryName(projectPath);

            // Run the migration
            var process = Process.Start(new ProcessStartInfo
            {
                FileName = "dotnet",
                Arguments = $"ef database update --connection \"{connectionString}\"",
                WorkingDirectory = workingDir,
                RedirectStandardOutput = true,
                RedirectStandardError = true
            });

            await process.WaitForExitAsync(cancellationToken);
            if (process.ExitCode != 0)
            {
                var error = await process.StandardError.ReadToEndAsync();
                throw new InvalidOperationException($"Migration failed for {migrationResource.Name}: {error}");
            }

            // Optionally update the resource state in the dashboard
            // (You'd need to use the resource notification service, which is more advanced)
        }
    }
}
```

To register this lifecycle hook, add it to the builder in `Program.cs`:

```csharp
builder.Services.AddLifecycleHook<MigrationLifecycleHook>();
```

Now, when the AppHost runs, after all resources are created (containers started, etc.), this hook will execute and run migrations.

#### 9.3.4 Setting Up References on the Migration Resource

We need to configure the migration resource to reference both the database and the project. Update the extension method to accept these references:

```csharp
public static IResourceBuilder<MigrationResource> AddMigration(
    this IDistributedApplicationBuilder builder,
    string name,
    IResourceBuilder<PostgresDatabaseResource> database,
    IResourceBuilder<ProjectResource> project)
{
    var resource = new MigrationResource(name);
    var resourceBuilder = builder.AddResource(resource)
        .WithReference(database)
        .WithReference(project)
        .WithInitialState(new CustomResourceSnapshot
        {
            ResourceType = "Migration",
            State = "Pending",
            Properties = []
        });

    return resourceBuilder;
}
```

Now in `Program.cs`:

```csharp
var migration = builder.AddMigration("migration", productsDb, apiService);
```

Note: `productsDb` is a `PostgresDatabaseResource`, and `apiService` is a `ProjectResource`. The `WithReference` calls add `ReferenceAnnotation` to the migration resource, which our lifecycle hook can find.

#### 9.3.5 Handling the Resource State

We’d also like the dashboard to show the migration as “Running” while it’s executing, and then “Completed” or “Failed”. To update the state dynamically, we need to use the `ResourceNotificationService`. This is more advanced, but let's sketch it.

In the lifecycle hook, you can inject `ResourceNotificationService` and send updates:

```csharp
public class MigrationLifecycleHook : IDistributedApplicationLifecycleHook
{
    private readonly ResourceNotificationService _notificationService;

    public MigrationLifecycleHook(ResourceNotificationService notificationService)
    {
        _notificationService = notificationService;
    }

    public async Task AfterResourcesCreatedAsync(DistributedApplicationModel appModel, CancellationToken cancellationToken = default)
    {
        // ... find migration resources ...
        foreach (var migrationResource in migrationResources)
        {
            await _notificationService.PublishUpdateAsync(migrationResource, state => state with
            {
                State = "Running"
            });

            try
            {
                // run migration ...
                await _notificationService.PublishUpdateAsync(migrationResource, state => state with
                {
                    State = "Completed"
                });
            }
            catch
            {
                await _notificationService.PublishUpdateAsync(migrationResource, state => state with
                {
                    State = "Failed"
                });
                throw;
            }
        }
    }
}
```

But `ResourceNotificationService` might not be available at that point? Actually, it is a scoped service, but lifecycle hooks are singletons. You can use `IServiceProvider` to create a scope. We'll skip full implementation here, but the pattern exists.

---

### 9.4 Running Custom Setup Scripts

The migration resource is an example of a one‑time task. You can generalize this pattern to run any script: seeding data, calling external APIs, etc. You would create a custom resource type and a lifecycle hook that executes the script.

For example, a `SeedDataResource` that runs a seed script after the database is migrated.

---

### 9.5 Using WithAnnotation for Metadata

Annotations are key‑value pairs attached to resources. They can hold any metadata you need. For example, you might annotate a resource with a version number, or with a flag indicating whether it should be exposed publicly.

You can add an annotation using `WithAnnotation<T>`:

```csharp
var apiService = builder.AddProject<Projects.MyAspireApp_ApiService>("apiservice")
    .WithAnnotation(new EndpointAnnotation(ProtocolType.Http, uriScheme: "http", name: "public"))
    .WithAnnotation(new MyCustomAnnotation { SomeValue = 123 });
```

Then later, in a lifecycle hook or elsewhere, you can retrieve annotations:

```csharp
var customAnnotation = resource.Annotations.OfType<MyCustomAnnotation>().FirstOrDefault();
```

Annotations are powerful for extending the resource model without creating new resource types.

---

### 9.6 Hands‑on: Create a Migration Resource for EF Core

Now let's implement the migration resource in our `MyAspireApp` solution. We’ll use Entity Framework Core with PostgreSQL.

#### Step 1: Add EF Core to the API Service

In the API service project, install EF Core and the Npgsql provider:

```bash
cd MyAspireApp.ApiService
dotnet add package Microsoft.EntityFrameworkCore.Design
dotnet add package Npgsql.EntityFrameworkCore.PostgreSQL
```

Create a `DbContext` and entity models. For simplicity, we'll reuse the `Product` and `Order` classes but set up EF Core.

Create `AppDbContext.cs`:

```csharp
using Microsoft.EntityFrameworkCore;

namespace MyAspireApp.ApiService.Data;

public class AppDbContext : DbContext
{
    public AppDbContext(DbContextOptions<AppDbContext> options) : base(options) { }

    public DbSet<Product> Products { get; set; }
    public DbSet<Order> Orders { get; set; }
    public DbSet<OrderItem> OrderItems { get; set; }

    protected override void OnModelCreating(ModelBuilder modelBuilder)
    {
        modelBuilder.Entity<Product>().ToTable("Products");
        modelBuilder.Entity<Order>().ToTable("Orders");
        modelBuilder.Entity<OrderItem>().ToTable("OrderItems");
    }
}
```

Update `Program.cs` to register EF Core with the connection string. But note: we want migrations to be run by the migration resource, so we need to ensure the connection string is available. We'll register `DbContext` after obtaining the connection string.

In `Program.cs`, replace the raw `NpgsqlDataSource` registration with EF Core:

```csharp
// Build connection string from configuration (as we did in Chapter 7)
var dbHost = builder.Configuration["Database:Host"] ?? "localhost";
var dbPort = builder.Configuration["Database:Port"] ?? "5432";
var dbName = builder.Configuration["Database:Name"] ?? "shop";
var dbUser = builder.Configuration["Database:User"] ?? "postgres";
var dbPassword = builder.Configuration["Database:Password"];

var connectionString = $"Host={dbHost};Port={dbPort};Database={dbName};Username={dbUser};Password={dbPassword}";

builder.Services.AddDbContext<AppDbContext>(options =>
    options.UseNpgsql(connectionString));
```

Now, instead of raw SQL, use the DbContext in endpoints. For example:

```csharp
app.MapGet("/products", async (AppDbContext db) =>
    await db.Products.ToListAsync());
```

#### Step 2: Add Migration Tools and Create Initial Migration

In the API service project, add a reference to the EF Core tools (if not already) and create a migration:

```bash
dotnet add package Microsoft.EntityFrameworkCore.Tools
dotnet ef migrations add InitialCreate
```

This creates a `Migrations` folder. We want our migration resource to run `dotnet ef database update` against this project.

#### Step 3: Implement the Custom Resource in AppHost

Follow the steps outlined earlier:

- Create `MigrationResource` class.
- Create extension method `AddMigration` that takes database and project references.
- Create `MigrationLifecycleHook` that runs `dotnet ef database update` with the connection string.
- Register the lifecycle hook in `Program.cs`.

Let's write the code in full.

**MigrationResource.cs**

```csharp
using Aspire.Hosting.ApplicationModel;

namespace MyAspireApp.AppHost;

public class MigrationResource : Resource
{
    public MigrationResource(string name) : base(name)
    {
    }
}
```

**MigrationResourceBuilderExtensions.cs**

```csharp
using Aspire.Hosting.ApplicationModel;

namespace Aspire.Hosting;

public static class MigrationResourceBuilderExtensions
{
    public static IResourceBuilder<MigrationResource> AddMigration(
        this IDistributedApplicationBuilder builder,
        string name,
        IResourceBuilder<PostgresDatabaseResource> database,
        IResourceBuilder<ProjectResource> project)
    {
        var resource = new MigrationResource(name);
        return builder.AddResource(resource)
            .WithReference(database)
            .WithReference(project)
            .WithInitialState(new CustomResourceSnapshot
            {
                ResourceType = "Migration",
                State = "Pending",
                Properties = []
            });
    }
}
```

**MigrationLifecycleHook.cs**

```csharp
using System.Diagnostics;
using Aspire.Hosting.ApplicationModel;
using Aspire.Hosting.Lifecycle;

namespace MyAspireApp.AppHost;

public class MigrationLifecycleHook(ResourceNotificationService notificationService) : IDistributedApplicationLifecycleHook
{
    public async Task AfterResourcesCreatedAsync(DistributedApplicationModel appModel, CancellationToken cancellationToken = default)
    {
        var migrationResources = appModel.Resources.OfType<MigrationResource>().ToList();
        if (migrationResources.Count == 0)
            return;

        foreach (var migrationResource in migrationResources)
        {
            await notificationService.PublishUpdateAsync(migrationResource, state => state with { State = "Running" });

            try
            {
                // Get database reference
                var dbReference = migrationResource.Annotations.OfType<ReferenceAnnotation>()
                    .FirstOrDefault(r => r.Resource is PostgresDatabaseResource);
                if (dbReference == null)
                {
                    throw new InvalidOperationException($"Migration resource {migrationResource.Name} has no database reference.");
                }
                var dbResource = (PostgresDatabaseResource)dbReference.Resource;
                var connectionString = await dbResource.GetConnectionStringAsync(cancellationToken);

                // Get project reference
                var projectReference = migrationResource.Annotations.OfType<ReferenceAnnotation>()
                    .FirstOrDefault(r => r.Resource is ProjectResource);
                if (projectReference == null)
                {
                    throw new InvalidOperationException($"Migration resource {migrationResource.Name} has no project reference.");
                }
                var projectResource = (ProjectResource)projectReference.Resource;
                var projectPath = projectResource.GetProjectMetadata().ProjectPath;
                var workingDir = Path.GetDirectoryName(projectPath);

                // Run EF Core migration
                var process = new Process
                {
                    StartInfo = new ProcessStartInfo
                    {
                        FileName = "dotnet",
                        Arguments = $"ef database update --connection \"{connectionString}\"",
                        WorkingDirectory = workingDir,
                        RedirectStandardOutput = true,
                        RedirectStandardError = true,
                        UseShellExecute = false,
                        CreateNoWindow = true
                    }
                };

                process.Start();
                await process.WaitForExitAsync(cancellationToken);

                if (process.ExitCode != 0)
                {
                    var error = await process.StandardError.ReadToEndAsync(cancellationToken);
                    throw new InvalidOperationException($"Migration failed: {error}");
                }

                await notificationService.PublishUpdateAsync(migrationResource, state => state with { State = "Completed" });
            }
            catch (Exception ex)
            {
                await notificationService.PublishUpdateAsync(migrationResource, state => state with
                {
                    State = "Failed",
                    Properties = state.Properties.Add(new ResourcePropertySnapshot(CustomResourceKnownProperties.Source, "error") { Value = ex.Message })
                });
                throw;
            }
        }
    }
}
```

**Register in AppHost Program.cs**

```csharp
using MyAspireApp.AppHost; // for MigrationResource

var builder = DistributedApplication.CreateBuilder(args);

// Add services for lifecycle hook
builder.Services.AddLifecycleHook<MigrationLifecycleHook>();

// Existing resources
var postgres = builder.AddPostgres("postgres").WithDataVolume();
var productsDb = postgres.AddDatabase("productsdb");

var apiService = builder.AddProject<Projects.MyAspireApp_ApiService>("apiservice")
    .WithReference(productsDb)
    .WithReference(messaging) // if you have messaging
    .WithReference(blobs);    // if you have blobs

var migration = builder.AddMigration("migration", productsDb, apiService);

// Ensure API depends on migration (so API starts after migration completes)
apiService.WithReference(migration); // This creates a dependency

// ... other resources (web frontend, etc.)

builder.Build().Run();
```

**Important**: The `WithReference(migration)` on the API service creates a dependency, so the API will wait for the migration to be in a running state. But our migration resource transitions from Pending → Running → Completed. The API will start only after the migration is "running". However, our migration runs in `AfterResourcesCreatedAsync`, which happens after all resources are created. The API resource will be in a "Starting" state while the migration runs. The dependency will ensure the API does not start until the migration is considered "running". Since our migration quickly completes, the API will start after.

We also need to ensure that the database container is ready before the migration runs. That's already true because the migration runs after resources are created (including the database container). However, the database might take a few seconds to be ready. We could add a health check or wait for it. For simplicity, we assume it's ready.

#### Step 4: Test

Run the AppHost. Observe the dashboard. You should see the `migration` resource appear, transition from "Pending" to "Running" to "Completed". The API service will start only after the migration is completed. If you check the database, the tables should be created.

---

### 9.7 Other Lifecycle Hooks

Lifecycle hooks aren't just for one‑time tasks. You can use `BeforeStartAsync` to validate configuration, create directories, or even modify the application model before any resources are launched. For example, you could add environment variables to all projects based on some condition.

---

### 9.8 Summary

In this chapter, you learned how to extend .NET Aspire by creating custom resources and using lifecycle hooks. Key takeaways:

- **Custom resources** allow you to model any component of your application, not just the built‑in ones.
- **Annotations** (like `WithReference`, `WithAnnotation`) attach metadata to resources, which can be retrieved later.
- **Lifecycle hooks** (`BeforeStart`, `AfterResourcesCreated`) let you run code at specific points in the application startup.
- You built a practical `MigrationResource` that runs EF Core migrations before the API starts, ensuring the database schema is up‑to‑date.

These techniques give you the power to adapt Aspire to any complex scenario. In the next chapter, we’ll dive into **Advanced Service Discovery and Resiliency**, where you’ll learn how to fine‑tune service communication, implement custom load balancing, and use advanced Polly policies.

---

**Exercises**

1. Modify the migration resource to wait for the database to be healthy before running migrations (hint: use `IResourceBuilder.WaitFor` or a health check check).
2. Create a custom resource that seeds the database with initial data after migrations complete.
3. Add a lifecycle hook that logs all environment variables injected into each project (for debugging purposes).
4. Explore the `ResourceNotificationService` and update the migration resource’s state with a progress message.

In Chapter 10, we’ll explore **Advanced Service Discovery and Resiliency** to make your microservices even more robust.